In [ ]:
# class Node:
#     def __init__(self, node_id, name, action_id):
#         self.id = node_id
#         self.name = name
#         self.action_id = action_id
#         self.routes = {}  # Dictionary of route IDs to following node IDs
# 
#     def add_route(self, route_id, following_node_id, rule):
#         self.routes[route_id] = (following_node_id, rule)
# 
#     def get_following_node(self, route_id: int):
#         if route_id in self.routes:
#             return self.routes[route_id]
#         else:
#             return None


In [ ]:
# class Action:
#     def __init__(self, action_id, name, action_type, data):
#         self.id = action_id
#         self.name = name
#         self.type = action_type
#         self.data = data

In [ ]:
# class Flow:
#     def __init__(self):
#         self.nodes = {}  # Dictionary of node IDs to Node objects
#         self.actions = {}  # Dictionary of action IDs to Action objects
#         self.routes = {}  # Dictionary of route IDs to (from_node_id, to_node_id, rule) tuples
#         self.next_node_id = 1
#         self.next_action_id = 1
#         self.next_route_id = 1
# 
#     def add_node(self, name, action_type, action_data):
#         node_id = f"N{self.next_node_id}"
#         action_id = f"A{self.next_action_id}"
#         self.next_node_id += 1
#         self.next_action_id += 1
#         action = Action(action_id, name, action_type, action_data)
#         self.actions[action_id] = action
#         node = Node(node_id, name, action_id)
#         self.nodes[node_id] = node
#         return node
# 
#     def add_route(self, from_node, to_node, rule):
#         route_id = f"R{self.next_route_id}"
#         self.next_route_id += 1
#         self.routes[route_id] = (from_node.id, to_node.id, rule)
#         from_node.add_route(route_id, to_node.id, rule)
# 
#     def edit_route(self, from_node, route_id, new_to_node, new_rule):
#         if route_id in from_node.routes:
#             from_node.routes[route_id] = (new_to_node.id, new_rule)
# 
#     def delete_route(self, from_node, route_id):
#         if route_id in from_node.routes:
#             del from_node.routes[route_id]
# 
#     def delete_node(self, node_id):
#         if node_id in self.nodes:
#             node = self.nodes[node_id]
#             for other_node in self.nodes.values():
#                 for route_id, (following_id, _) in node.routes.items():
#                     if following_id == node_id:
#                         other_node.delete_route(route_id)
#             del self.nodes[node_id]
# 
#     def encode_flow_to_json(self):
#         data = {
#             "nodes": {node.id: {"name": node.name, "action_id": node.action_id, "routes": node.routes} for node in self.nodes.values()},
#             "actions": {action.id: {"name": action.name, "type": action.type, "data": action.data} for action in self.actions.values()},
#             "routes": {route_id: (from_id, to_id, rule) for route_id, (from_id, to_id, rule) in self.routes.items()}
#         }
#         return json.dumps(data)
# 
#     @staticmethod
#     def decode_flow_from_json(json_data):
#         data = json.loads(json_data)
#         flow = Flow()
# 
#         for node_id, node_data in data["nodes"].items():
#             action_id = node_data["action_id"]
#         node = Node(node_id, node_data["name"], action_id)
#         flow.nodes[node_id] = node
#         flow.actions[action_id] = flow.actions.get(action_id)  # Avoid creating duplicate actions
# 
#         for route_id, route_data in data["routes"].items():
#             print(route_id, route_data)
#             flow.add_route(route_id, route_data[1], route_data[2])
# 
#         return flow


In [ ]:
# flow = Flow()

In [ ]:
# flow.add_node("Start", "start", {})
# flow.add_node("node 2", "action", {"action": "action 1"})
# flow.add_route(flow.nodes["N1"], flow.nodes["N2"], "x > 5")
# flow.add_route(flow.nodes["N2"], flow.nodes["N1"], "x <= 5")

In [ ]:
# json_text = flow.encode_flow_to_json()
# json_text

In [ ]:
# decoded_flow = Flow.decode_flow_from_json(json_text)

## Simplified (old ver):

In [ ]:
import ast
import json
from typing import Any, Literal


class Node:
    def __init__(self, node_id: int, action_id: int, name):
        self.id = node_id
        self.name = name
        self.action_id = action_id
        self.routes = {}

    @staticmethod
    def from_dict(node_id: int, dict_data: dict[str, Any]):
        node = Node(node_id, dict_data["action_id"], dict_data["name"])
        node.routes = dict_data["routes"]
        return node

    def to_dict(self):
        return {"name": self.name, "action_id": self.action_id, "routes": self.routes}

    def add_route(self, route_id: int, to_node_id: int, rule):  # rule can be a function for later
        self.routes[route_id] = (to_node_id, rule)

    def get_route(self, route_id: int):
        return self.routes.get(route_id)


class Action:
    def __init__(self, action_id: int, name, action_type, data):
        self.id = action_id
        self.name = name
        self.type = action_type
        self.data = data

    @staticmethod
    def from_dict(action_id: int, data: dict[str, Any]):
        return Action(action_id, data["name"], data["type"], data["data"])

    def to_dict(self):
        return {"name": self.name, "type": self.type, "data": self.data}


class Flow:
    def __init__(self):
        self.nodes: dict[int, Node] = {}  # Dictionary of node IDs to Node objects
        self.actions: dict[int, Action] = {}  # Dictionary of action IDs to Action objects
        self.next_node_id = 1
        self.next_action_id = 1
        self.next_route_id = 1

    def add_action(self, action_name, action_type, action_data):
        action = Action(self.next_action_id, action_name, action_type, action_data)
        self.actions[action.id] = action
        self.next_action_id += 1
        return action

    def edit_action(self, action_id: int, new_name, new_type, new_data):
        action = self.actions.get(action_id)
        assert action is not None, "Action not found"
        action.name = new_name
        action.type = new_type
        action.data = new_data

    def delete_action(self, action_id: int):
        self.actions.pop(action_id, None)

    def add_node(self, node_name, action_id: int):
        node = Node(self.next_node_id, action_id, node_name)
        self.nodes[node.id] = node
        self.next_node_id += 1
        return node

    def add_node_with_new_action(self, node_name, action_name, action_type, action_data):
        action = self.add_action(action_name, action_type, action_data)
        return self.add_node(node_name, action.id)

    def edit_node(self, node_id: int, new_name, new_action_id: int):
        node = self.nodes.get(node_id)
        assert node is not None, "Node not found"
        assert new_action_id in self.actions, "Invalid action ID"
        node.name = new_name
        node.action_id = new_action_id

    def add_route(self, from_node_id: int, to_node_id: int, rule):
        assert from_node_id in self.nodes and to_node_id in self.nodes, "Invalid node ID"
        self.nodes[from_node_id].add_route(self.next_route_id, to_node_id, rule)
        self.next_route_id += 1

    def edit_route(self, from_node_id: int, route_id: int, new_to_node_id: int, new_rule):
        assert from_node_id in self.nodes and new_to_node_id in self.nodes, "Invalid node ID"
        route = self.nodes[from_node_id].get_route(route_id)
        assert route is not None, "Route not found"
        self.nodes[from_node_id].routes[route_id] = (new_to_node_id, new_rule)

    def delete_route(self, from_node_id: int, route_id: int):
        route = self.nodes[from_node_id].get_route(route_id)
        assert route is not None, "Route not found"
        del self.nodes[from_node_id].routes[route_id]

    def delete_node(self, node_id: int):
        node = self.nodes.get(node_id)
        assert node is not None, "Node not found"
        for other_node in self.nodes.values():
            for route_id, (following_id, _) in node.routes.items():
                if following_id == node_id:
                    other_node.delete_route(route_id)
        del self.nodes[node_id]

    def serialize(self, output_type: Literal["json", "python"] = "json"):
        data = {
            "nn": self.next_node_id,
            "na": self.next_action_id,
            "nr": self.next_route_id,
            "nodes": {node.id: node.to_dict() for node in self.nodes.values()},
            "actions": {action.id: action.to_dict() for action in self.actions.values()},
        }
        return json.dumps(data) if output_type == "json" else str(data)

    @staticmethod
    def deserialize(serialized_data: str, input_type: Literal["json", "python"] = "json"):
        data = json.loads(serialized_data) if input_type == "json" else ast.literal_eval(serialized_data)
        flow = Flow()
        flow.next_node_id = data["nn"]
        flow.next_action_id = data["na"]
        flow.next_route_id = data["nr"]
        for node_id_key, node_data in data["nodes"].items():
            node_id = int(node_id_key) if input_type == "json" else node_id_key
            flow.nodes[node_id] = Node.from_dict(node_id, node_data)
        for action_id_key, action_data in data["actions"].items():
            action_id = int(action_id_key) if input_type == "json" else action_id_key
            flow.actions[action_id] = Action.from_dict(action_id, action_data)
        return flow

    def print_readable(self):
        print("Nodes:")
        for node in self.nodes.values():
            print(f"\tN{node.id}: {node.name} (Action ID: A{node.action_id})")
            print(f"\tRoutes:")
            for route_id, (to_node_id, rule) in node.routes.items():
                print(f"\t\tR{route_id}: Go to node {to_node_id} (Rule: {rule})")
        print("Actions:")
        for action in self.actions.values():
            print(f"\tA{action.id}: \n\t\tType: {action.type}\n\t\tData: {action.data}")


In [ ]:
test_flow = Flow()
action1 = test_flow.add_action("Startup action", "prewritten", "hello abcasdgt sdyasdgly yasd24 atyasv60 adhbvc asdr5y12")
test_flow.add_node("Startup", action1.id)
test_flow.add_node_with_new_action("Search for item", "Embedding search", "run-command", {"item": "bla bla"})
test_flow.add_route(1, 2, "if user logged in")
action3 = test_flow.add_action("Run bot", "run-chat", "asdyft1346 aayvm4 613u asdfladsfh 35asdfylasdc asjlspz")
test_flow.add_node("Chat with bot", action3.id)
test_flow.add_route(1, 3, "if user responded")
test_flow.add_route(3, 2, "if user press some button")
test_flow.delete_route(3, 1)

test_flow.print_readable()

In [ ]:
import time

now = time.time()
serialized_flow_python = test_flow.serialize(output_type="python")
first = time.time()
serialized_flow_json = test_flow.serialize()
second = time.time()

print("Python:", first - now)
print("JSON:", second - first)

print(serialized_flow_python)
print("")
print(serialized_flow_json)

In [ ]:
now = time.time()
deserialized_flow_python = Flow.deserialize(serialized_flow_python, input_type="python")
first = time.time()
deserialized_flow_json = Flow.deserialize(serialized_flow_json)
second = time.time()

print("Python:", first - now)
print("JSON:", second - first)

deserialized_flow_python.print_readable()
print("")
deserialized_flow_json.print_readable()

In [ ]:
a = """Nodes:
	N1: Startup (Action ID: A1)
	Routes:
		R1: Go to node 2 (Rule: if user logged in)
		R2: Go to node 3 (Rule: if user responded)
	N2: Search for item (Action ID: A2)
	Routes:
	N3: Chat with bot (Action ID: A3)
	Routes:
		R3: Go to node 2 (Rule: if user press some button)
Actions:
	A1: 
		Type: prewritten
		Data: hello abcasdgt sdyasdgly yasd24 atyasv60 adhbvc asdr5y12
	A2: 
		Type: run-command
		Data: {'item': 'bla bla'}
	A3: 
		Type: run-chat
		Data: asdyft1346 aayvm4 613u asdfladsfh 35asdfylasdc asjlspz
"""
b = """Nodes:
	N1: Startup (Action ID: A1)
	Routes:
		R1: Go to node 2 (Rule: if user logged in)
		R2: Go to node 3 (Rule: if user responded)
	N2: Search for item (Action ID: A2)
	Routes:
	N3: Chat with bot (Action ID: A3)
	Routes:
		R3: Go to node 2 (Rule: if user press some button)
Actions:
	A1: 
		Type: prewritten
		Data: hello abcasdgt sdyasdgly yasd24 atyasv60 adhbvc asdr5y12
	A2: 
		Type: run-command
		Data: {'item': 'bla bla'}
	A3: 
		Type: run-chat
		Data: asdyft1346 aayvm4 613u asdfladsfh 35asdfylasdc asjlspz
"""
a == b

In [ ]:
from pprint import pprint

pprint(
    {'nn': 4, 'na': 4, 'nr': 4, 'nodes': {1: {'name': 'Startup', 'action_id': 1, 'routes': {1: (2, 'if user logged in'), 2: (3, 'if user responded')}},
                                          2: {'name': 'Search for item', 'action_id': 2, 'routes': {}},
                                          3: {'name': 'Chat with bot', 'action_id': 3, 'routes': {3: (2, 'if user press some button')}}},
     'actions': {1: {'name': 'Startup action', 'type': 'prewritten', 'data': 'hello abcasdgt sdyasdgly yasd24 atyasv60 adhbvc asdr5y12'},
                 2: {'name': 'Embedding search', 'type': 'run-command', 'data': {'item': 'bla bla'}},
                 3: {'name': 'Run bot', 'type': 'run-chat', 'data': 'asdyft1346 aayvm4 613u asdfladsfh 35asdfylasdc asjlspz'}}}, sort_dicts=False)